In [1]:
import boto3
import time
import json

session = boto3.Session(profile_name='nikita')
client = session.client('logs')


import numpy as np
import pandas as pd
import math
%pylab inline


from datetime import datetime, timezone

Populating the interactive namespace from numpy and matplotlib


# Query results

In [2]:
start = 0
results = []

while True:
    print('Querying')
    
    # Query for the first time begining in 1970
    query = client.start_query(
        logGroupName='/aws/batch/job',
        startTime=start,
        endTime=int(time.time()),
        queryString="""
            fields @message, @timestamp
            | filter success == 1
            | sort @timestamp asc
            | limit 10000
        """,
        limit=10000
    )

    # Loop untill the query response is ready
    query_result = {'status': 'Scheduled'}
    while query_result['status'] in ('Scheduled', 'Running'):
        query_result = client.get_query_results(queryId=query['queryId'])
        time.sleep(1)
    
    tmp = query_result
    
    # If we reached the end, we don't have any more data, we break
    if len(query_result['results']) == 0:
        break

    # If we did not reach the end, we get the timestamp of the last line and use this
    # as a starting point for the next iteration
    assert query_result['results'][-1][1]['field'] == "@timestamp"
    start = int(datetime.strptime(query_result['results'][-1][1]['value'], "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo=timezone.utc).timestamp()) + 1

    # Process the data a little bit and push it to results
    data = [
        json.loads(row[0]['value'])
        for row in query_result['results']
    ]
    results += data

print(f"Done, with {len(results)} results")

Querying
Querying
Done, with 6122 results


In [7]:
df.side

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
6117      0
6118      0
6119      1
6120      1
6121      0
Name: side, Length: 6122, dtype: object

In [3]:
for row in data:
    if row['winner'] is None:
        row['winner'] = float('+inf')

df = pd.DataFrame(data)

In [4]:
from datetime import timedelta as td

class Player:
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        
class GreedyPlayer(Player):
    pass

class MCTSPlayer(Player):
    pass

class UCTPlayer(Player):
    pass

# Fix bad data
df['version'] = df.version.fillna(1).astype(int)
df['side'] = df['side'].f.astype(int)

# Only keep usable data
df = df[df.version >= 2]

ValueError: cannot convert float NaN to integer

In [ ]:
df.to_pickle("raw-results.pkl")

In [ ]:
df.sample(5)

In [ ]:
is_normalized = df['side'] == 0

to_normalise = df[~is_normalized].copy()

# Swap columns
opponents = to_normalise['opponent'].copy()
players = to_normalise['player'].copy()

to_normalise['player'] = opponents
to_normalise['opponent'] = players

# Swap winner
to_normalise['winner'] = (to_normalise['winner'] + 1) % 2

# Swap individual scores
to_normalise['score'] = to_normalise['score'].map(lambda x: list(reversed(x)))

# Concatenate the 2 sides
normalised = pd.concat([
    df[is_normalized].copy(),
    to_normalise.copy()
])

# Fix the ties
normalised[normalised['winner'] == float('inf')] = 0.5

# Evaluate both players
normalised['player_eval'] = normalised['player'].map(lambda x: eval(x))
normalised['opponent_eval'] = normalised['opponent'].map(lambda x: eval(x))

# Plot

## UTC c=1.5

In [ ]:
uct = normalised[normalised.pool == 'uct-1_5-vs-range'].copy()
uct['c'] = uct.opponent_eval.map(lambda x: x.kwargs['c'])
grouped = uct.groupby('c').winner.agg(['mean', 'count'])

In [ ]:
plt.figure(figsize=(10, 6))

plt.scatter(
    grouped.index,
    grouped['mean'],
    c=grouped['count'],
    label='win ratio',
    s=100
)

plt.vlines(1.5, 0, 1, linestyles='dashed', label="$1.5$", color="grey")
plt.hlines(0.5, grouped.index.min() , grouped.index.max(), linestyles='dotted', label="0.5 win ratio", color="grey")
plt.legend()
cb = plt.colorbar()
cb.set_label('# of matches', rotation=270)
#cb.ax.set_yticklabels([int(i) for i in cb.get_ticks()])
cb.ax.get_yaxis().labelpad = 15

plt.ylim(0, 1)

plt.title("Win ratios of different $c$ values against $c = 1.5$");
plt.xlabel("Opponent's value of $c$")
plt.ylabel("Opponent's win ratio");